# Joint Hypocenter-Velocity Inversion

Note: most of the following is coming from the thesis of Lisa Block (1991)

The nonlinear joint hypocenter-velocity inversion problem is solved by iterating the linearized problem, following the separation of parameters technique described by Pavlis and Booker (1980).  Three types of constraints are applied to the velocity models. To prevent wild fluctuations in the velocity structures at poorly resolved nodes, a smoothness constraint is added. This is accomplished by minimizing the spatial velocity derivatives in addition to the arrivai time residuals. Another constraint is applied to keep the velocities within specified bounds. It is implemented by applying a penalty for velocities which fall outside the desired range, and then minimizing the penalties during the subsequent iteration. With the last type of velocity constraint, velocities computed from independent surveys are used to directly constrain the velocity model at certain points within the grid.

## Formulation

Let $t_o$ be an observed arrivai time (either P or S); $t_c$ = the calculated arrivai time, based on initial estimates of the velocity structure, earthquake location and origin time, and station correction; and $r$ the residual, i.e. $r - t_o - t_c$.  The goal is to change the model parameters so that the new calculated arrivai time, $t_c$ + $\Delta t_c$ is equal to the observed arrivai time

\begin{equation}
\Delta t_c = t_o - t_c = r.
\end{equation}

Expanding $\Delta t_c$ in terms of changes in the model parameters and keeping only the first order terms gives

\begin{equation}
\Delta t_0+\frac{\partial t}{\partial x}\Delta x+
    \frac{\partial t}{\partial y}\Delta y+
    \frac{\partial t}{\partial z}\Delta z+
    \sum_{j=1}^{nnodes} \frac{\partial t}{\partial v_j}\Delta v_j+\Delta sc = r,
\end{equation}
    
where $t_0$ is the earthquake origin time, $x$, $y$, and $z$ are the hypocenter coordinates, $v_j$ is the velocity of the $j^{th}$ node, $nnodes$ is the number of velocity nodes in the inversion grid, and $sc$ is the station correction. All of the observations for one earthquake, say for the $i^{th}$ earthquake, yield a set of equations which can be put into matrix form

\begin{equation}
\begin{bmatrix}
1 & \partial t_1 / \partial x_i & \partial t_1 / \partial y_i & \partial t_1 / \partial z_i \\
\cdot & \cdot & \cdot & \cdot \\
\cdot & \cdot & \cdot & \cdot \\
\cdot & \cdot & \cdot & \cdot \\
1 & \partial t_{narr} / \partial x_i & \partial t_{narr} / \partial y_{narr} & \partial t_{narr} / \partial z_i
\end{bmatrix}  
\begin{bmatrix}
\Delta t_{0i} \\
\Delta x_i \\
\Delta y_i \\
\Delta z_i
\end{bmatrix} +
\end{equation}

\begin{equation}
\begin{bmatrix}
\partial t_1 / \partial v_1^P & - & \partial t_1 / \partial v_{nnodes}^P & \partial t_1 / \partial v_1^S & - & \partial t_1 / \partial v_{nnodes}^S & \partial t_1 / \partial sc_i^P & - & \partial t_1 / \partial sc_{nsta}^P & \partial t_1 / \partial sc_1^S & - & \partial t_1 / \partial sc_{nsta}^S \\
\cdot & - & \cdot & \cdot & - & \cdot & \cdot & - & \cdot & \cdot & - & \cdot \\
\cdot & - & \cdot & \cdot & - & \cdot & \cdot & - & \cdot & \cdot & - & \cdot \\
\cdot & - & \cdot & \cdot & - & \cdot & \cdot & - & \cdot & \cdot & - & \cdot \\
\partial t_{narr} / \partial v_1^P & - & \partial t_{narr} / \partial v_{nnodes}^P & \partial t_{narr} / \partial v_1^S & - & \partial t_{narr} / \partial v_{nnodes}^S & \partial t_{narr} / \partial sc_i^P & - & \partial t_{narr} / \partial sc_{nsta}^P & \partial t_{narr} / \partial sc_1^S & - & \partial t_{narr} / \partial sc_{nsta}^S
\end{bmatrix}
\begin{bmatrix}
\Delta v_1^P \\
| \\
\Delta v_{nnodes}^P \\
\Delta v_1^S \\
| \\
\Delta v_{nnodes}^S \\
\Delta sc_1^P \\
| \\
\Delta sc_{nsta}^P \\
\Delta sc_1^S \\
| \\
\Delta sc_{nsta}^S
\end{bmatrix}
\end{equation}

\begin{equation}
= 
\begin{bmatrix}
r_1 \\
\cdot \\
\cdot \\
\cdot \\
r_{narr}
\end{bmatrix},
\end{equation}

or more concisely by

\begin{equation}
\mathbf{H}_i \Delta  \mathbf{h}_i + \mathbf{M}_i \Delta \mathbf{m} = \mathbf{r}_i
\end{equation}

where $narr$ is the number of arrivai times recorded for the event, and $nsta$ is the number of stations. $\mathbf{M}_i$ is a sparse matrix.  For a particular row, the only nonzero partial derivatives with respect to velocity are those which correspond to nodes along the ray path for the appropriate phase. Only the station correction partial derivative corresponding to the appropriate station and phase is nonzero and has a value of +1.  For efficiency reasons, matrices $\mathbf{M}_i$ are built at the raytracing step, in the C++ routine, and passed along with traveltime and raypath data.

For an inversion which finds the $V_S/V_P$ ratios, rather than the S wave velocities, the partial derivatives of the S wave travel times with respect to the S wave velocities are replaced by the partial derivatives of the S wave travel times with respect to the $V_S/V_P$ ratios. Also, in this case, the partial derivatives of the S wave travel times with respect to the P wave velocities are nonzero. These issues are discussed in more detail further below.

## Partial Derivatives

### Velocity Partial Derivatives


A seismic travel time is approximated by a discrete sum. Dividing the ray path into $n$ segments of length $\Delta s$, the travel time is given by
\begin{equation}
t = \sum_{l=1}^n\frac{\Delta s}{v(x_l,y_l,z_l)},
\end{equation}
where $v(x_l, y_l, z_l)$ is the velocity at the center of the $l^{th}$ segment. The partial derivative of the travel time with respect to the $j^{th}$ velocity node, $v_j$, is therefore
\begin{equation}
\frac{\partial t}{\partial v_j} = \sum_{l=1}^n\frac{-1}{v(x_l,y_l,z_l)^2}\frac{\partial v(x_l,y_l,z_l)}{\partial v_j}\Delta s.
\end{equation}

For each ray segment, $\partial v(x_l,y_l,z_l)/\partial v_j$ is only nonzero for the eight velocity nodes sur-
rounding it and is found by differentiating an inverse distance interpolation expression:
\begin{equation}
\frac{\partial v(x_l,y_l,z_l)}{\partial v_j} = \left(1-\frac{|x_l-x_i|}{dx}\right)\left(1-\frac{|y_l-y_i|}{dy}\right)\left(1-\frac{|z_l-z_i|}{dz}\right).
\end{equation}
The last two equations are used to compute partial derivatives of P wave travel times with respect to P wave velocities, and partial derivatives of S wave travel times with respect to S wave velocities. These are the only velocity partial derivatives required for an inversion which finds $V_P$ and $V_S$.

For an inversion which finds $V_P$ and $V_S/V_P$, three types of velocity derivatives are required. The first type is the partial derivative of the P wave travel time with respect to the P wave velocities and is given by the equations above. The other two types are the partial derivatives of the S wave travel time with respect to the P wave velocities and the $V_S/V_P$ ratios. These derivatives can be found by using the chain rule of elementary calculus. Consider the following expression for the $j^{th}$ S wave velocity:
\begin{equation}
v_j^S = v_j^P\left(\frac{V_S}{V_P}\right)_j.
\end{equation}
Applying the chain rule to the above equation gives:
\begin{equation}
\frac{\partial t^S}{\partial(V_S/V_P)_j} = \frac{\partial t^S}{\partial v_j^S}\frac{\partial v_j^S}{\partial(V_S/V_P)_j} = \frac{\partial t^S}{\partial v_j^S} v_j^P,
\end{equation}
and
\begin{equation}
\frac{\partial t^S}{\partial v_j^P} = \frac{\partial t^S}{\partial v_j^S} \frac{\partial v_j^S}{\partial v_j^P} = \frac{\partial t^S}{\partial v_j^S}\left(\frac{V_S}{V_P}\right)_j.
\end{equation}

Hence, the partial derivatives of the S wave travel times with respect to the P wave velocities and the $V_S/V_P$ ratios are found by scaling their partial derivatives with respect to the S wave velocities.

### Hypocenter Partial Derivatives

Thurber (1981) showed that the partial derivatives of the arrivai time with respect to the hypocenter coordinates $(x, y, z)$ are approximated by

\begin{equation}
\frac{\partial t}{\partial x} = \frac{-1}{v(x,y,z)}\frac{dx}{ds}
\end{equation}

\begin{equation}
\frac{\partial t}{\partial y} = \frac{-1}{v(x,y,z)}\frac{dy}{ds}
\end{equation}

\begin{equation}
\frac{\partial t}{\partial z} = \frac{-1}{v(x,y,z)}\frac{dz}{ds}
\end{equation}

where $dx/ds$, $dy/ds$, and $dz/ds$ are the direction cosines of the ray path at the hypocenter
location, and $v(x,y, z)$ is the velocity at the hypocenter.

## Separation of Parameters

Recall that $\mathbf{H}_i$ is the hypocenter partial derivative matrix for the $i^{th}$ earthquake. If there are more than 4 observed arrivai times, then there exists a matrix $\mathbf{T}_i$ such that $\mathbf{T}_i^T\mathbf{H}_i = 0$ (Pavlis and Booker, 1980; Roecker, 1982). This can be seen by considering the singular value decomposition of $\mathbf{H}_i$:

\begin{equation}
\mathbf{H}_i = \mathbf{U}_i\begin{bmatrix}
\underline{
\begin{array}
 \lambda\lambda_1 & 0 & 0 & 0 \\
0 & \lambda_2 & 0 & 0 \\
0 & 0 & \lambda_3 & 0 \\
0 & 0 & 0 & \lambda_4
\end{array}} \\
\\
0
\end{bmatrix}\mathbf{V}_i^T
\end{equation}

$\mathbf{U}_i$ and $\mathbf{V}_i$ are orthonormal matrices with dimensions $narr \times narr$ and $4 \times 4$, respectively, and the matrix of singular values has dimensions $narr \times 4$. Since the last $(narr-4)$ columns of $\mathbf{U}_i$ multiply only zeros, the equation may be rewritten:

\begin{equation}
\mathbf{H}_i = \mathbf{U}'_i\begin{bmatrix}
\lambda_1 & 0 & 0 & 0 \\
0 & \lambda_2 & 0 & 0 \\
0 & 0 & \lambda_3 & 0 \\
0 & 0 & 0 & \lambda_4
\end{bmatrix}\mathbf{V}_i^T
\end{equation}

where $\mathbf{U}'_i$ consists of the first 4 columns of $\mathbf{U}_i$. Now, let $\mathbf{U}''_i$ equal the part of $\mathbf{U}_i$ which was eliminated. Since $\mathbf{U}_i$ is orthonormal, ${\mathbf{U}''_i}^T\mathbf{U}'_i=0$, and therefore ${\mathbf{U}''_i}^T\mathbf{H}_i=0$. Thus, the $\mathbf{T}_i$ matrix which we seek is simply equivalent to $\mathbf{U}''_i$.  Multipying $\mathbf{H}_i \Delta  \mathbf{h}_i + \mathbf{M}_i \Delta \mathbf{m} = \mathbf{r}_i$ by $\mathbf{T}_i^T$ thus gives

\begin{equation}
\mathbf{T}_i^T\mathbf{M}_i \Delta \mathbf{m} = \mathbf{T}_i^T\mathbf{r}_i
\end{equation}

or

\begin{equation}
\mathbf{M}'_i \Delta \mathbf{m} = \mathbf{r}'_i.
\end{equation}

This procedure is done for each earthquake and then the results are combined into one matrix equation:

\begin{equation}
\mathbf{M}'\Delta \mathbf{m} = 
\begin{bmatrix}
\mathbf{M}'_1 \\
\mathbf{M}'_2 \\
| \\
\mathbf{M}'_{nev}
\end{bmatrix} \Delta \mathbf{m} = 
\begin{bmatrix}
\mathbf{r}'_1 \\
\mathbf{r}'_2 \\
| \\
\mathbf{r}'_{nev}
\end{bmatrix} = 
\mathbf{r}'.
\end{equation}

When shot data are incorporated into the inversion, the separation of parameters step is skipped, since the locations of the shots are known. The partial derivatives and residuals for the shots are added directly into the last equation.

## Constraints

### Regularization

An easy and effective constraint to apply is to require the velocity models to be smooth using the method of regularization. This constraint prevents wild fluctuations in the velocity structures due to poorly resolved nodes. In addition to minimizing the arrivai time residuals, the spatial derivatives of the P wave and S wave velocity structures are also minimized.

In the current implementation, second derivatives are used.  Depending on the position if the node with respect to the edge of the grid, the following forward, central, or backward operator are used:

\begin{equation}
\text{central operator}\qquad f''(x) = \frac{f(x+h)-2f(x)+f(x-h)}{h^2}
\end{equation}

\begin{equation}
\text{forward operator}\qquad f''(x) = \frac{f(x+2h)-2f(x+h)+f(x)}{h^2}
\end{equation}

\begin{equation}
\text{backward operator}\qquad f''(x) = \frac{f(x)-2f(x-h)+f(x-2h)}{h^2}
\end{equation}

where $h$ is the grid step size.  The derivatives are computed independently along $x$, $y$ and $z$, and three matrices are thus built ($\mathbf{K}_x$, $\mathbf{K}_y$ and $\mathbf{K}_z$), one for each spatial dimention.  Consider three consecutive nodes along $x$, $v_{i-1}$, $v_i$, $v_{i-1}$, where index $i$ stands for location along $x$. After 1 iteration these velocities will be perturbed to $v_{i-1} +\Delta v_{i-1}$, $v_i +\Delta v_i$, and $v_{i+1} +\Delta  v_{i+1}$.  The second derivative of the new velocity along $x$ is (central operator)

\begin{equation}
\left(\frac{1}{h^2}\right)\left(v_{i-1} +\Delta v_{i-1}\right) +
\left(\frac{2}{h^2}\right)\left(v_i +\Delta v_i\right) +
\left(\frac{1}{h^2}\right)\left(v_{i+1} +\Delta v_{i+1}\right)
\end{equation}

To simplify notation, let $a = 1/h^2$ and $b=2/h^2$.  Rearranging terms, we get

\begin{equation}
(a v_{i-1} - b v_i + a v_{i+1}) + (a \Delta v_{i-1} - b \Delta v_i + a \Delta v_{i+1}) = \text{constant} + (a\, -\!b\;\; a) 
\begin{pmatrix}
\Delta v_{i-1} \\
\Delta v_i \\
\Delta v_{i+1}
\end{pmatrix}.
\end{equation}

Similar expressions are constructed for each group of three consecutive nodes in the $x$, $y$, and $z$ directions, for both the P wave and S wave velocity structures (or the $V_s/V_p$ structure).  All of the derivative expressions are incorporated into one matrix equation

\begin{equation}
\text{Spatial derivatives} = \mathbf{c}_x + \mathbf{c}_y + \mathbf{c}_z + \mathbf{K}_x \begin{bmatrix}
\Delta v_1^P \\
| \\
\Delta v_{nnodes}^P \\
\Delta v_1^S \\
| \\
\Delta v_{nnodes}^S \\
\end{bmatrix} + \mathbf{K}_y \begin{bmatrix}
\Delta v_1^P \\
| \\
\Delta v_{nnodes}^P \\
\Delta v_1^S \\
| \\
\Delta v_{nnodes}^S \\
\end{bmatrix} + \mathbf{K}_z \begin{bmatrix}
\Delta v_1^P \\
| \\
\Delta v_{nnodes}^P \\
\Delta v_1^S \\
| \\
\Delta v_{nnodes}^S \\
\end{bmatrix},
\end{equation}

where vectors $\mathbf{c}$ contain the spatial derivatives of the current velocity models, and matrices $\mathbf{K}$ are recangular matrices containing the $a$'s and $b$'s in the appropriate columns.  By adding ($2 \times nsta$) columns of zeros to matrices $\mathbf{K}$, the latter equation can be expressed in terms of the solution vector $\Delta\mathbf{m}$:

\begin{eqnarray}
\text{Spatial derivatives} & = & \mathbf{c}_x + \mathbf{c}_y + \mathbf{c}_z + \left[ \mathbf{K}_x \; |\; 0 \right] \Delta\mathbf{m} + \left[ \mathbf{K}_y \; |\; 0 \right] \Delta\mathbf{m} + \left[ \mathbf{K}_z \; |\; 0 \right] \Delta\mathbf{m} \\
& = & \mathbf{c}_x + \mathbf{c}_y + \mathbf{c}_z + \mathbf{K}'_x\Delta\mathbf{m} + \mathbf{K}'_y\Delta\mathbf{m} + \mathbf{K}'_z\Delta\mathbf{m}
\end{eqnarray}

Keeping three separate $\mathbf{K}$ matrices allows weighting independently the smoothing constraint in $x$, $y$ and $z$, as shown further below.

### Velocity Penalty Function

In addition to requiring the velocity models to be smooth, a constraint is also applied to keep the P wave and S wave velocity values within specified ranges. This constraint is implemented by computing a penalty for each velocity which falls outside the desired range and then minimizing these penalties. A simple linear penalty function works well

\begin{equation}
  P(v)=\left\{\begin{array}{l l l}
    A(v-v_{max}) & \quad\text{if}\quad & v>v_{max}\\
    0            & \quad\text{if}\quad & v_{min} \le v \le v_{max}\\
    A(v_{min}-v) & \quad\text{if}\quad & v<v_{min}\end{array}\right.
\end{equation}

Naturally, the velocity bounds will be different for P wave velocities and S wave velocities (or $V_S/V_P$ ratios), and therefore there are actually two different penalty functions, $P_P(v)$ and $P_S(v)$. For simplicity, the subscripts $P$ and $S$ are dropped in the following description.

Consider one velocity, $v_i$, and the penalty associated with it, $P(v_i)$. When the velocity is perturbed, the new penalty $P(v_i + \Delta v_i)$ will be $P(v_i) + \left( \frac{\partial P(v_i)}{\partial v_i}\right)\Delta v_i$. The penalties of all of the perturbed velocities can be put into a matrix expression

\begin{eqnarray}
\text{Penalties} & = & \begin{bmatrix}
P(v_1^P) \\
| \\
P(v_{nnodes}^P) \\
P(v_1^S) \\
| \\
P(v_{nnodes}^S)
\end{bmatrix} + 
\begin{bmatrix}
\partial P(v_1^P)/\partial v_1^P & & & & \\
 & \cdot & & & 0 \\
 & & \cdot & & \\
 0 & & & \cdot & \\
 & & & & \partial P(v_{nnodes}^S)/\partial v_{nnodes}^S
 \end{bmatrix} 
 \begin{bmatrix}
\Delta v_1^P \\
| \\
\Delta v_{nnodes}^P \\
\Delta v_1^S \\
| \\
\Delta v_{nnodes}^S \\
\end{bmatrix}\\
 & = & \mathbf{p} + \partial \mathbf{P}\begin{bmatrix}
\Delta v_1^P \\
| \\
\Delta v_{nnodes}^P \\
\Delta v_1^S \\
| \\
\Delta v_{nnodes}^S \\
\end{bmatrix}
\end{eqnarray}

As with the regularization constraint, the above expression may be expressed in terms of the solution vector $\Delta\mathbf{m}$ by appending ($2 \times nsta$) columns of zeros to $\partial\mathbf{P}$:

\begin{eqnarray}
\text{Penalties} & = & \mathbf{p} + \left[ \partial\mathbf{P} \; |\; 0 \right] \Delta\mathbf{m} \\
& = & \mathbf{p} + \partial\mathbf{P}' \Delta\mathbf{m}
\end{eqnarray}


### Velocity Data Points

Interval velocities computed from other surveys can used to directly constrain the velocities at certain points within the grid. For example, suppose that the velocity at the point $(X, Y, Z)$ is found from seismic refl.ection or VSP results to be equal to V. Then, in the velocity inversion, in addition to minimizing the arrivai time residuals and the other constraint terms previously described, we will also require the velocity at the point (X, Y, Z) to be equal to, or at least close to, $V$. The constraint is formulated as follows. If the current value of the velocity at the point (X, Y, Z) is v, we want to perturb the velocity such that the new velocity is equal to $V$

\begin{equation}
v_{new}(X,Y,Z) = v(X,Y,Z) + \Delta v(X,Y,Z) = V.
\end{equation}

Because the velocity at any point within the grid is a linear interpolation of the velocity values at the eight surrounding nodes, we can express the latter in terms of the velocities and velocity perturbations at the nodes, i.e.

\begin{equation}
\sum_{i=1}^8\left(1-\frac{|X-x_i|}{dx}\right)\left(1-\frac{|Y-y_i|}{dy}\right)\left(1-\frac{|Z-z_i|}{dz}\right)(v_i+\Delta v_i) = V.
\end{equation}

One such equation is constructed for each velocity data point, and all of the equations can be combined in one matrix equation

\begin{equation}
\mathbf{D}\begin{bmatrix}
(v_1+\Delta v_1)^P \\
| \\
(v_{nnodes} + \Delta v_{nnodes})^P
\end{bmatrix} = 
\begin{bmatrix}
V_1 \\
| \\
V_{npts}
\end{bmatrix}.
\end{equation}

The above equation is confined to P wave velocity constraints since usually only P wave data are available. The matrix $\mathbf{D}$ contains the interpolation weighting terms in the appropriate columns. The dimensions of $\mathbf{D}$ are $npts \times nnodes$, where $npts$ is the number of velocity data points, and $nnodes$ is the number of velocity nodes in the grid. (The method can be extended to S wave velocity constraints, or $V_S/V_P$ constaints, by adding additional rows to the above eqation, and adding additional columns to $\mathbf{D}$.) This equation can be simplified by taking the terms involving the "current" velocity values, $v_i$, to the righthand side:

\begin{equation}
\mathbf{D}\begin{bmatrix}
(\Delta v_1)^P \\
| \\
(\Delta v_{nnodes})^P
\end{bmatrix} = 
\begin{bmatrix}
V_1 \\
| \\
V_{npts}
\end{bmatrix} - \mathbf{D}\begin{bmatrix}
(v_1)^P \\
| \\
(v_{nnodes})^P
\end{bmatrix} = \mathbf{g}.
\end{equation}

In order to write this equation in terms of the solution vector, columns of zeros are added to $\mathbf{D}$

\begin{equation}
\left[\mathbf{D} \; |\; 0 \right] \Delta\mathbf{m} = \mathbf{D}' \Delta\mathbf{m} = \mathbf{g}.
\end{equation}


### Origin Times - Station Corrections Nonuniqueness Constraint

This final constraint deals with an inherent nonuniqueness of the inversion when all of the station corrections are allowed to vary. In this case, there is a trade-off between event origin times and station corrections. All of the origin times could be shifted by a constant factor, and all of the station corrections could be shifted by the same factor, and the residuals would not change.

To compensate for this inherent nonuniqueness, the sum of the final P wave station corrections is required to be zero. This constraint is sufficient to eliminate the nonuniqueness issue, even if S wave arrival times are also included in the inversion. The constraint is expressed mathematically as follows. Let $sc_{i}^P$ be the "current" P wave station correction at the $i^{th}$ station, and let $\Delta sc_i^P$ be the perturbation to that station correction during one iteration. Setting the sum of the new station corrections equal to zero gives

\begin{equation}
\sum_{i=1}^{nsta}(sc_i^P + \Delta sc_i^P) = 0
\end{equation}

or equivalently

\begin{equation}
\sum_{i=1}^{nsta}\Delta sc_i^P = - \sum_{i=1}^{nsta}sc_i^P.
\end{equation}

The summation on the right-hand side reduces to a known constant. To simplify the notation, let this term be called $s$. Expressing the term on the left-hand side in vector form, the equation becomes

\begin{equation}
\begin{bmatrix}
1 & \ldots & 1
\end{bmatrix}
\begin{bmatrix}
\Delta sc_1^P \\
| \\
\Delta sc_{nsta}^P
\end{bmatrix}
 = -s,
\end{equation}

or

\begin{equation}
\mathbf{u}^T
\begin{bmatrix}
\Delta sc_1^P \\
| \\
\Delta sc_{nsta}^P
\end{bmatrix}
 = -s,
\end{equation}

where $\mathbf{u}$ is a vector containing $nsta$ elements, each equal to 1. By adding ($2 \times nnodes$) zeros to the beginning of $\mathbf{u}$ (corresponding to the P wave and S wave velocity nodes) and $nsta$ zeros to the end of $\mathbf{u}$ (corresponding to the S wave station corrections), the latter equation can be expressed in terms of the solution vector

\begin{equation}
\left[ 0\,\, | \,\, \mathbf{u} \,\, | \,\, 0\right]^T \Delta\mathbf{m} = {\mathbf{u}'}^T \Delta\mathbf{m} = -s.
\end{equation}



## The Constrained Least Squares Solution

The constrained least squares solution is found by minimizing

\begin{multline}
\sum(\text{arrival time residuals})^2 + \lambda\sum(\text{spatial derivatives})^2 \\
+ \gamma\sum(\text{penalties})^2 + \alpha\sum(\text{velocity data point misfits})^2 \\
+\sum(\text{P wave station correction sum})^2,
\end{multline}

which is expressed mathematically by

\begin{multline}
\left(\mathbf{M}'\Delta \mathbf{m}-\mathbf{r}'\right)^T \left(\mathbf{M}'\Delta \mathbf{m}-\mathbf{r}'\right) 
+ \lambda_x\left(\mathbf{c}_x + \mathbf{K}'_x\Delta \mathbf{m}\right)^T \left(\mathbf{c}_x+\mathbf{K}'_x\Delta \mathbf{m}\right) \\
+ \lambda_y\left(\mathbf{c}_y + \mathbf{K}'_y\Delta \mathbf{m}\right)^T \left(\mathbf{c}_y+\mathbf{K}'_y\Delta \mathbf{m}\right)
+ \lambda_z\left(\mathbf{c}_z + \mathbf{K}'_z\Delta \mathbf{m}\right)^T \left(\mathbf{c}_z+\mathbf{K}'_z\Delta \mathbf{m}\right) 
+ \gamma\left(\mathbf{p}+\partial\mathbf{P}'\Delta \mathbf{m}\right)^T\left(\mathbf{p}+\partial\mathbf{P}'\Delta \mathbf{m}\right) \\
\alpha\left(\mathbf{D}'\Delta\mathbf{m} - \mathbf{g}\right)^T\left(\mathbf{D}'\Delta\mathbf{m} - \mathbf{g}\right)
+ \left({\mathbf{u}'}^T \Delta \mathbf{m} + s\right)^T\left({\mathbf{u}'}^T \Delta \mathbf{m} + s\right).
\end{multline}

$\lambda$, $\gamma$ and $\alpha$ are Lagrangian multipliers. A small value of $\lambda$ will yield small arrivai time residuals but rough velocity structures, and a large value of $\lambda$ will yield very smooth velocity structures but larger residuals. The appropriate value to use depends on how much velocity variation is reasonable from a geologie point of view and must be found by experimentation. Similarly, the larger the value of $\gamma$, the more strongly the velocities are constrained within the defined ranges; and the larger the value of $\alpha$, the more strongly the velocities are constrained to match the values obtained from other surveys. A Lagrangian multiplier is not required for the term involving the P wave station correction sum because this constraint compensates for an inherent nonuniqueness in the inversion. In other words, this constraint can be exactly satisfied without affecting the other terms in the equation. The minimum of the latter expression is found by setting the partial derivatives with respect to each model parameter perturbation $\Delta m_i$ equal to 0.  The solution is given by

\begin{multline}
\Delta \mathbf{m} = 
	\left( \underbrace{{\mathbf{M}'}^T\mathbf{M}'+
	       \lambda_x{\mathbf{K}'_x}^T\mathbf{K}'_x+
	       \lambda_y{\mathbf{K}'_y}^T\mathbf{K}'_y+
	       \lambda_z{\mathbf{K}'_z}^T\mathbf{K}'_z+
            \gamma\partial{\mathbf{P}'}^T\partial\mathbf{P}'+
            \alpha{\mathbf{D}'}^T\mathbf{D}' + 
		\mathbf{u}'{\mathbf{u}'}^T }_{\displaystyle\mathbf{A}}
  	\right)^{-1} \\
     \left({\mathbf{M}'}^T \mathbf{r}'+\lambda_x{\mathbf{K}'_x}^T \mathbf{c}_x+\lambda_y{\mathbf{K}'_y}^T \mathbf{c}_y+\lambda_z{\mathbf{K}'_z}^T \mathbf{c}_z+
        \gamma\partial{\mathbf{P}'}^T \mathbf{p}+\alpha{\mathbf{D}'}^T\mathbf{g} + \mathbf{u}'s\right).
\end{multline}

In the current implementation, $\lambda_x=\lambda_y=\lambda$ and $\lambda_z = w_z\lambda$, i.e. a vertical to horizontal smoothing ratio is applied.  Matrices $\mathbf{M}'$, $\mathbf{K}'$, $\partial\mathbf{P}'$, and $\mathbf{D}'$ are large and sparse, such that $\mathbf{A}$ is sparse as well.  The lsqr algorithm is used for the inversion to take advantage of the matrix sparsity.

## Hypocenter Relocation

The hypocenters are relocated using the revised velocity models and station corrections by the simple undamped least squares method. Since the problem is nonlinear, the process is iterative. The sum of the squares of the new arrival time residuals after one iteration will be (to first order)

\begin{equation}
\sum(\text{new arrival time residuals})^2 = (\mathbf{H}_i\Delta \mathbf{h}_i - \mathbf{r}_i)^T
    (\mathbf{H}_i\Delta \mathbf{h}_i - \mathbf{r}_i)
\end{equation}

where $\mathbf{H}_i$ is the hypocenter partial derivative matrix for the $i^{th}$ event which was described above, $\Delta\mathbf{h}_i$ contains the perturbations to the hypocenter coordinates and origin time, and $\mathbf{r}_i$ contains the current arrival time residuals. The minimum is given by

\begin{equation}
 \Delta\mathbf{h}_i = (\mathbf{H}_i^T\mathbf{H}_i)^{-1}\mathbf{H}_i^T \mathbf{r}_i.
\end{equation}

This is the conventional undamped least squares solution. The hypocenter coordinates and origin time are updated, and the inversion is repeated until either the perturbations to the hypocenter parameters become smaller than some specified limits or the maximum number of iterations is reached.  If the latter system of equation is ill conditionned, an SVD solution is employed.

Recall that the partial derivatives in the matrix $\mathbf{H}_i$ depend on the current event location. Therefore, the algorithm is stable if the corrections $\Delta\mathbf{h}_i$ are small. Following the advice of Roecker (1982), the solution is found progressively, in order to minimize the effects of nonlinearity. Specifically, the hypocenter latitude and longitude are allowed to vary first, since they are almost always the most stable parameters. After these two parameters have converged, the depth and origin time are also allowed to vary - that is, all four parameters are now varying. The reason for solving the inversion progressively is that by allowing the two most stable parameters to vary first, the residuals are reduced. This means that the sizes of the corrections $\Delta\mathbf{h}_i$ that occur when the less stable parameters are allowed to vary have probably been decreased, thereby reducing the effects of nonlinearity.